In [21]:
import pandas as pd
import numpy as np

In [35]:
train_data = pd.read_csv("Dataset/Train.csv")
test_data = pd.read_csv("Dataset/Test.csv")
sample_sub = pd.read_csv("Dataset/SampleSubmission.csv")

In [36]:
print("train shape :", train_data.shape)
print("test shape : ", test_data.shape)

train shape : (8523, 12)
test shape :  (5681, 11)


In [37]:
y_train = np.log10(train_data.Item_Outlet_Sales)
X_train = train_data.drop('Item_Outlet_Sales', axis=1)
X_test = test_data
X = pd.concat([X_train, X_test])

In [38]:
categoricals = X_train.select_dtypes(include='object').columns
numericals = X_train.select_dtypes(exclude='object').columns
print(f'{len(categoricals)} categorical features')
print(f'{len(numericals)} numerical features')

7 categorical features
4 numerical features


In [39]:
X[categoricals].isna().sum().sort_values(ascending=False)

Outlet_Size             4016
Outlet_Type                0
Outlet_Location_Type       0
Outlet_Identifier          0
Item_Type                  0
Item_Fat_Content           0
Item_Identifier            0
dtype: int64

In [40]:
X[categoricals] = X[categoricals].fillna("absent")

In [41]:
X[numericals].isna().sum().sort_values(ascending=False)

Item_Weight                  2439
Outlet_Establishment_Year       0
Item_MRP                        0
Item_Visibility                 0
dtype: int64

In [42]:
X = X.drop(["Item_Identifier","Outlet_Identifier"], axis = 1)

In [43]:
X = pd.get_dummies(X)

In [44]:
X[numericals] = X[numericals].fillna(X[numericals].mean())

In [47]:
X_train = X[0:8523]
X_test = X[8523:]

In [48]:
from sklearn import linear_model
#regr = linear_model.Lasso(alpha=0.1)
regr = linear_model.LinearRegression()
regr = regr.fit(X_train, y_train)
out = regr.predict(X_test)

out = 10**out
raw_data = {'Item_Identifier' : sample_sub["Item_Identifier"],
            'Outlet_Identifier' : sample_sub["Outlet_Identifier"],
            'Item_Outlet_Sales' : out}
result_ = pd.DataFrame(raw_data)
result_.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1431.368493
1,FDW14,OUT017,1086.153358
2,NCN55,OUT010,592.283807
3,FDQ58,OUT017,2118.712759
4,FDY38,OUT027,6376.320197


In [49]:
result_.to_csv("submission.csv",index=False)